In [45]:
import pandas as pd
import numpy as np
from  tensorflow.compat.v1 import keras
import tensorflow.compat.v1.keras.backend as K
from tensorflow.compat.v1.keras import Model
from tensorflow.compat.v1.keras import initializers, regularizers, constraints
from tensorflow.compat.v1.keras.layers import *
from tensorflow.compat.v1.keras.layers import Embedding, Dense, CuDNNLSTM,CuDNNGRU, Bidirectional,SpatialDropout1D,Input,GlobalAveragePooling1D,GlobalMaxPooling1D,Conv1D,concatenate,Dropout,Activation,BatchNormalization,Concatenate,Add,MaxPooling1D,Flatten,AveragePooling1D
from gensim.models import Word2Vec
from tensorflow.compat.v1.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.compat.v1.keras.preprocessing.sequence import pad_sequences
from tensorflow.compat.v1.keras.preprocessing import text, sequence
import gensim, logging
from tensorflow.compat.v1.keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import KFold,StratifiedShuffleSplit,StratifiedKFold
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
train=pd.read_csv('../tcdata/train.csv',header=None)
test=pd.read_csv('../tcdata/track1_round1_testB.csv',header=None)
# test=pd.read_csv('../tcdata/testA.csv',header=None)
data=pd.concat([train,test],axis=0)
data[1]=data[1].apply(lambda x:x.strip().replace('|',''))

In [3]:
## 制作标签
### 创建训练集标签 
train_labels1=np.zeros((len(train),17),dtype='int8')
noncase=0
for cnt,i in enumerate(train[2]):
    if i:
        lab=[int(x.replace('|','').strip()) for x in i.split(' ') if x and x!='|']
        for l in lab:
            train_labels1[cnt,l]=1
    else:
        noncase+=1
print('noncase label1:',noncase)
#----------------------------------
noncase=0
train_labels2=np.zeros((len(train),12),dtype='int8')
for cnt,i in enumerate(train[3]):
    if pd.notna(i):
        lab=[int(x.replace('|','').strip()) for x in i.split(' ') if x and x!='|']
        for l in lab:
            train_labels2[cnt,l]=1
    else:
        noncase+=1
print('noncase label2:',noncase)

noncase label1: 0
noncase label2: 8247


In [4]:
cate_num=train_labels1.sum(1)+train_labels2.sum(1)

In [4]:
## 生成全部的训练文本
data1=pd.read_csv('../tcdata/track1_round1_train_20210222.csv',header=None)
data2=pd.read_csv('../tcdata/track1_round1_testA_20210222.csv',header=None)
data3=pd.read_csv('../tcdata/track1_round1_testB.csv',header=None)
text_data=pd.concat([train,data1,data2,data3],axis=0)
text_data[1]=text_data[1].apply(lambda x:x.strip().replace('|',''))
text_data[1].to_csv('../tmp/all_data.txt',header=False,index=False)

In [12]:
text_data=pd.read_csv('../tmp/all_data.txt',header=None)

In [5]:
# w2v=Word2Vec(text_data[0].apply(lambda x:x.split(' ')).tolist(),size=128, window=8, iter=30, min_count=2,
#                      sg=1, sample=0.002, workers=6 , seed=1017)

# w2v.wv.save_word2vec_format('../tmp/w2v_128.txt')

In [5]:
tokenizer = Tokenizer(lower=False, char_level=False, split=' ')
tokenizer.fit_on_texts(data[1].tolist())
seq = tokenizer.texts_to_sequences(data[1].tolist())
# 分训练和测试集合
seq = pad_sequences(seq, maxlen=128, value=0)
train_seq=np.asarray(seq[:len(train)])
test_seq=seq[len(train):]

In [6]:
embedding_matrix=np.zeros((len(tokenizer.word_index)+1,128))
w2v=gensim.models.KeyedVectors.load_word2vec_format(
        '../tmp/w2v_128.txt', binary=False)

for word in tokenizer.word_index:
    if word not in w2v.wv.vocab:
        continue
    embedding_matrix[tokenizer.word_index[word]] = w2v[word]
embedding_matrix.shape

2021-04-21 16:37:57,955 : INFO : loading projection weights from ../tmp/w2v_128.txt
2021-04-21 16:37:58,068 : INFO : loaded (859, 128) matrix from ../tmp/w2v_128.txt
<ipython-input-6-6203a5121046>:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word not in w2v.wv.vocab:


(859, 128)

In [7]:
def NN_huaweiv1(maxlen,embedding_matrix=None,class_num1=17,class_num2=12):    
    emb_layer = Embedding(
       embedding_matrix.shape[0], embedding_matrix.shape[1],input_length=maxlen,weights=[embedding_matrix],trainable=False,
    )
    seq1 = Input(shape=(maxlen,)) 
    
    x1 = emb_layer(seq1)
    sdrop=SpatialDropout1D(rate=0.2)
    lstm_layer = Bidirectional(CuDNNGRU(128, return_sequences=True))
    gru_layer = Bidirectional(CuDNNGRU(128, return_sequences=True))
    cnn1d_layer=Conv1D(64, kernel_size=3, padding="same", kernel_initializer="he_uniform")
    x1 = sdrop(x1)
    lstm1 = lstm_layer(x1)
    gru1 = gru_layer(lstm1)
    att_1 = Attention(maxlen)(lstm1)
    att_2 = Attention(maxlen)(gru1)
    cnn1 = cnn1d_layer(lstm1)

    avg_pool = GlobalAveragePooling1D()
    max_pool = GlobalMaxPooling1D()

    x1=concatenate([att_1,att_2,Attention(maxlen)(cnn1),avg_pool(cnn1),max_pool(cnn1)])

    x = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(128)(x1))))
    x = Activation(activation="relu")(BatchNormalization()(Dense(64)(x)))
    pred1 = Dense(class_num1, activation='sigmoid',name='pred1')(x)
    y=concatenate([x1,x])
    y = Activation(activation="relu")(BatchNormalization()(Dense(64)(x)))
    pred2=Dense(class_num2, activation='sigmoid',name='pred2')(y)
    
    model = Model(inputs=seq1, outputs=[pred1,pred2])
    return model

class AttentionSelf(Layer):
    """
        self attention,
        codes from:  https://mp.weixin.qq.com/s/qmJnyFMkXVjYBwoR_AQLVA
    """
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super().__init__(**kwargs)

    def build(self, input_shape):
        # W、K and V
        self.kernel = self.add_weight(name='WKV',
                                        shape=(3, input_shape[2], self.output_dim),
                                        initializer='uniform',
                                        regularizer=regularizers.L1L2(0.0000032),
                                        trainable=True)
        super().build(input_shape)

    def call(self, x):
        WQ = K.dot(x, self.kernel[0])
        WK = K.dot(x, self.kernel[1])
        WV = K.dot(x, self.kernel[2])
        # print("WQ.shape",WQ.shape)
        # print("K.permute_dimensions(WK, [0, 2, 1]).shape",K.permute_dimensions(WK, [0, 2, 1]).shape)
        QK = K.batch_dot(WQ, K.permute_dimensions(WK, [0, 2, 1]))
        QK = QK / (64**0.5)
        QK = K.softmax(QK)
        # print("QK.shape",QK.shape)
        V = K.batch_dot(QK, WV)
        return V

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.output_dim)

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
#         print('-------------',type(input_shape))
        self.W = self.add_weight(name='{}_W'.format(self.name),
                                 shape=(input_shape[-1],),
                                 initializer=self.init,
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(name='{}_b'.format(self.name),
                                     shape=(input_shape[1],),
                                     initializer='zero',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim
def mean_pred(y_true, y_pred):
    return -K.mean(y_true*K.log(y_pred+1.e-7)+(1-y_true)*K.log(1-y_pred+1.e-7))*10

## HAN

In [8]:
def HANGraph(maxlen,embedding_matrix,class_num1=17,class_num2=12):
    
    """
        构建神经网络
    :param hyper_parameters:json,  hyper parameters of network
    :return: tensor, moedl
    """

    emb_layer = Embedding(
    embedding_matrix.shape[0], embedding_matrix.shape[1],input_length=maxlen,weights=[embedding_matrix],trainable=False,
)
    seq1 = Input(shape=(maxlen,)) 
    x1 = emb_layer(seq1)
    # char or word
    x1=SpatialDropout1D(rate=0.2)(x1)
    x_word = word_level(maxlen)(x1)
    x_word_to_sen = Dropout(0.2)(x_word)

    # sentence or doc
    x_sen = sentence_level(maxlen)(x_word_to_sen)
    x_sen = Dropout(0.2)(x_sen)

    x_sen = Attention(128)(x_sen)
    # 最后就是softmax
    pred1 = Dense(class_num1, activation='sigmoid',name='pred1')(x_sen)
    pred2=Dense(class_num2, activation='sigmoid',name='pred2')(x_sen)
    output = [pred1,pred2]
    model = Model(seq1, output)
    return model

def word_level(maxlen):
    x_input_word = Input(shape=(maxlen,128))
    # x = SpatialDropout1D(self.dropout_spatial)(x_input_word)
    x = Bidirectional(CuDNNLSTM(units=128,
                          return_sequences=True,
                         ))(x_input_word)
    out_sent = AttentionSelf(128*2)(x)
    model = Model(x_input_word, out_sent)
    return model

def sentence_level(maxlen):
    x_input_sen = Input(shape=(maxlen, 128*2))
    # x = SpatialDropout1D(self.dropout_spatial)(x_input_sen)
    output_doc = Bidirectional(CuDNNLSTM(units=128*2,
                          return_sequences=True,
                          ))(x_input_sen)
    output_doc_att = AttentionSelf(128)(output_doc)
    model = Model(x_input_sen, output_doc_att)
    return model

##  textVDCNN

In [59]:
import tensorflow as tf
class VDCNNGraph():
    def __init__(self, maxlen,embedding_matrix,class_num1=17,class_num2=12):
        """
            初始化
        :param hyper_parameters: json，超参
        """
        self.maxlen=maxlen
        self.embedding_matrix=embedding_matrix
        self.class_num1=class_num1
        self.class_num2=class_num2
        self.dropout=0.2
        self.filters=[[64, 2], [128, 2], [256, 2], [512, 2]]
        self.pool_type= 'avg' # 池化选择, 可以选"max"、"avg"、"conv"
        self.l2=0
        self.activation_conv = 'linear'
        self.top_k=2
    
        self.shortcut = True

    def create_model(self,):
        """
            构建神经网络
        :param hyper_parameters:json,  hyper parameters of network
        :return: tensor, moedl
        """
        emb_layer = Embedding(
        self.embedding_matrix.shape[0], self.embedding_matrix.shape[1],input_length=self.maxlen,weights=[self.embedding_matrix],trainable=False,
)
        seq1 = Input(shape=(self.maxlen,)) 
        x1 = emb_layer(seq1)
        # char or word
        x1=SpatialDropout1D(rate=0.2)(x1)

        # 首先是 region embedding 层
        conv_1 = Conv1D(self.filters[0][0],
                        kernel_size=1,
                        strides=1,
                        padding='SAME',
                        kernel_regularizer=regularizers.l2(self.l2),
                        bias_regularizer=regularizers.l2(self.l2),
                        activation=self.activation_conv,
                        )(x1)
        block = ReLU()(conv_1)

        for filters_block in self.filters:
            for j in range(filters_block[1]-1):
                # conv + short-cut
                block_mid = self.convolutional_block(block, units=filters_block[0])
                block = shortcut_conv(block, block_mid, shortcut=True)
            # 这里是conv + max-pooling
            block_mid = self.convolutional_block(block, units=filters_block[0])
            block = shortcut_pool(block, block_mid, filters=filters_block[0], pool_type=self.pool_type, shortcut=True)

        block = k_max_pooling(top_k=self.top_k)(block)
        block = Flatten()(block)
        block = Dropout(self.dropout)(block)
        # 全连接层
        # block_fully = Dense(2048, activation='tanh')(block)
        # output = Dense(2048, activation='tanh')(block_fully)
        
        pred1 = Dense(self.class_num1, activation='sigmoid',name='pred1')(block)
        pred2=Dense(self.class_num2, activation='sigmoid',name='pred2')(block)
        output = [pred1,pred2]

        self.model = Model(inputs=seq1, outputs=output)
        self.model.summary(120)

    def convolutional_block(self, inputs, units=256):
        """
            Each convolutional block (see Figure 2) is a sequence of two convolutional layers, 
            each one followed by a temporal BatchNorm (Ioffe and Szegedy, 2015) layer and an ReLU activation. 
            The kernel size of all the temporal convolutions is 3, 
            with padding such that the temporal resolution is preserved 
            (or halved in the case of the convolutional pooling with stride 2, see below). 
        :param inputs: tensor, input
        :param units: int, units
        :return: tensor, result of convolutional block
        """
        x = Conv1D(units,
                    kernel_size=3,
                    padding='SAME',
                    strides=1,
                    kernel_regularizer=regularizers.l2(self.l2),
                    bias_regularizer=regularizers.l2(self.l2),
                    activation=self.activation_conv,
                    )(inputs)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv1D(units,
                    kernel_size=3,
                    strides=1,
                    padding='SAME',
                    kernel_regularizer=regularizers.l2(self.l2),
                    bias_regularizer=regularizers.l2(self.l2),
                    activation=self.activation_conv,
                    )(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        return x


def shortcut_pool(inputs, output, filters=256, pool_type='max', shortcut=True):
    """
        ResNet(shortcut连接|skip连接|residual连接), 
        这里是用shortcut连接. 恒等映射, block+f(block)
        再加上 downsampling实现
        参考: https://github.com/zonetrooper32/VDCNN/blob/keras_version/vdcnn.py
    :param inputs: tensor
    :param output: tensor
    :param filters: int
    :param pool_type: str, 'max'、'k-max' or 'conv' or other
    :param shortcut: boolean
    :return: tensor
    """
    if shortcut:
        conv_2 = Conv1D(filters=filters, kernel_size=1, strides=2, padding='SAME')(inputs)
        conv_2 = BatchNormalization()(conv_2)
        output = downsampling(output, pool_type=pool_type)
        out = Add()([output, conv_2])
    else:
        out = ReLU(inputs)
        out = downsampling(out, pool_type=pool_type)
    if pool_type is not None: # filters翻倍
        out = Conv1D(filters=filters*2, kernel_size=1, strides=1, padding='SAME')(out)
        out = BatchNormalization()(out)
    return out

def shortcut_conv(inputs, output, shortcut=True):
    """
        shortcut of conv
    :param inputs: tensor
    :param output: tensor
    :param shortcut: boolean
    :return: tensor
    """
    if shortcut:
        output = Add()([output, inputs])
    return output

def downsampling(inputs, pool_type='max'):
    """
        In addition, downsampling with stride 2 essentially doubles the effective coverage 
        (i.e., coverage in the original document) of the convolution kernel; 
        therefore, after going through downsampling L times, 
        associations among words within a distance in the order of 2L can be represented. 
        Thus, deep pyramid CNN is computationally efﬁcient for representing long-range associations 
        and so more global information. 
        参考: https://github.com/zonetrooper32/VDCNN/blob/keras_version/vdcnn.py
    :param inputs: tensor,
    :param pool_type: str, select 'max', 'k-max' or 'conv'
    :return: tensor,
    """
    if pool_type == 'max':
        output = MaxPooling1D(pool_size=3, strides=2, padding='SAME')(inputs)
    elif pool_type == 'k-max':
        output = k_max_pooling(top_k=int(K.int_shape(inputs)[1]/2))(inputs)
    elif pool_type == 'conv':
        output = Conv1D(kernel_size=3, strides=2, padding='SAME')(inputs)
    else:
        output = MaxPooling1D(pool_size=3, strides=2, padding='SAME')(inputs)
    return output

class k_max_pooling(Layer):
    """
        paper:        http://www.aclweb.org/anthology/P14-1062
        paper title:  A Convolutional Neural Network for Modelling Sentences
        Reference:    https://stackoverflow.com/questions/51299181/how-to-implement-k-max-pooling-in-tensorflow-or-keras
        动态K-max pooling
            k的选择为 k = max(k, s * (L-1) / L)
            其中k为预先选定的设置的最大的K个值，s为文本最大长度，L为第几个卷积层的深度（单个卷积到连接层等）
        github tf实现可以参考: https://github.com/lpty/classifier/blob/master/a04_dcnn/model.py
    """
    def __init__(self, top_k=8, **kwargs):
        self.top_k = top_k
        super().__init__(**kwargs)

    def build(self, input_shape):
        super().build(input_shape)

    def call(self, inputs):
        inputs_reshape = tf.transpose(inputs, perm=[0, 2, 1])
        pool_top_k = tf.nn.top_k(input=inputs_reshape, k=self.top_k, sorted=False).values
        pool_top_k_reshape = tf.transpose(pool_top_k, perm=[0, 2, 1])
        return pool_top_k_reshape

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.top_k, input_shape[-1]

In [60]:
batch_size=128
epochs=100
weight_name='V2'
oof=np.zeros((len(train),29))
tmp=0
test_oof=np.zeros((len(test),29))
# for ii in range(17):
#     per_labels=train_labels[:,ii]
#     print('当前分类类别：'+str(ii))
#     print('正样本比例:'+str(sum(per_labels)/len(per_labels)))
# per_labels=train_labels[:,1]
folds=StratifiedKFold(n_splits=8,shuffle=True, random_state=2018) #2018
for fold_n, (trn_idx, val_idx) in enumerate(folds.split(train,cate_num)):
    print('Fold', fold_n)
    print('Build model...')
#     print('正样本比例:',train_labels[trn_idx].mean(0))
#     model=NN_huaweiv1(maxlen=128,embedding_matrix=embedding_matrix)
#     model=HANGraph(128,embedding_matrix=embedding_matrix)
    vdcnn=VDCNNGraph(maxlen=128,embedding_matrix=embedding_matrix)
    vdcnn.create_model()
    model=vdcnn.model
#     model=multi_gpu_model(model,gpus=2)
    model.compile('adam', ['binary_crossentropy','binary_crossentropy'], metrics=['accuracy',mean_pred])

    print('Train...')
    early_stopping = EarlyStopping(monitor='val_loss', patience=2,mode='min')
    check_point=ModelCheckpoint('../model_weight/%s_%d.h5'%(weight_name,fold_n),monitor='val_loss',verbose=1, save_best_only=True,save_weights_only=True)

    model.fit(train_seq[trn_idx],{'pred1':train_labels1[trn_idx],'pred2':train_labels2[trn_idx]},
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping,check_point],
              validation_data=(train_seq[val_idx],{'pred1':train_labels1[val_idx],'pred2':train_labels2[val_idx]}))
    model.load_weights('../model_weight/%s_%d.h5'%(weight_name,fold_n))
    oof[val_idx,:17],oof[val_idx,17:] = model.predict(train_seq[val_idx],batch_size=batch_size)
    tmp_test_pred1,tmp_test_pred2=model.predict(test_seq,batch_size=batch_size)
    test_oof[:,:17]+=tmp_test_pred1/folds.n_splits
    test_oof[:,17:]+=tmp_test_pred2/folds.n_splits

/home/lichangyv/miniconda3/envs/tf2/lib/python3.8/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=8.
  warnings.warn(("The least populated class in y has only %d"


Fold 0
Build model...
Model: "model_10"
________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_16 (InputLayer)                  [(None, 128)]              0                                                     
________________________________________________________________________________________________________________________
embedding_12 (Embedding)               (None, 128, 128)           109952        input_16[0][0]                          
________________________________________________________________________________________________________________________
spatial_dropout1d_11 (SpatialDropout1D (None, 128, 128)           0             embedding_12[0][0]                      
_________________________________________________________________________________________________________________

137/137 [==============================] - 25s 129ms/step - loss: 0.4862 - pred1_loss: 0.2790 - pred2_loss: 0.2072 - pred1_accuracy: 0.1571 - pred1_mean_pred: 2.7895 - pred2_accuracy: 0.3578 - pred2_mean_pred: 2.0700 - val_loss: 0.3995 - val_pred1_loss: 0.2283 - val_pred2_loss: 0.1711 - val_pred1_accuracy: 0.3760 - val_pred1_mean_pred: 2.2795 - val_pred2_accuracy: 0.3044 - val_pred2_mean_pred: 1.7137

Epoch 00001: val_loss improved from inf to 0.39945, saving model to ../model_weight/V2_0.h5
Epoch 2/100
137/137 [==============================] - 17s 124ms/step - loss: 0.2342 - pred1_loss: 0.1505 - pred2_loss: 0.0837 - pred1_accuracy: 0.2044 - pred1_mean_pred: 1.5047 - pred2_accuracy: 0.4764 - pred2_mean_pred: 0.8370 - val_loss: 0.2724 - val_pred1_loss: 0.1562 - val_pred2_loss: 0.1163 - val_pred1_accuracy: 0.1072 - val_pred1_mean_pred: 1.5568 - val_pred2_accuracy: 0.4664 - val_pred2_mean_pred: 1.1642

Epoch 00002: val_loss improved from 0.39945 to 0.27243, saving model to ../model_weigh

137/137 [==============================] - 29s 158ms/step - loss: 0.4663 - pred1_loss: 0.2628 - pred2_loss: 0.2036 - pred1_accuracy: 0.1507 - pred1_mean_pred: 2.6274 - pred2_accuracy: 0.3742 - pred2_mean_pred: 2.0352 - val_loss: 0.3743 - val_pred1_loss: 0.2370 - val_pred2_loss: 0.1373 - val_pred1_accuracy: 0.1116 - val_pred1_mean_pred: 2.3651 - val_pred2_accuracy: 0.4756 - val_pred2_mean_pred: 1.3729

Epoch 00001: val_loss improved from inf to 0.37427, saving model to ../model_weight/V2_1.h5
Epoch 2/100
137/137 [==============================] - 21s 155ms/step - loss: 0.2259 - pred1_loss: 0.1448 - pred2_loss: 0.0811 - pred1_accuracy: 0.1924 - pred1_mean_pred: 1.4483 - pred2_accuracy: 0.4577 - pred2_mean_pred: 0.8109 - val_loss: 0.2481 - val_pred1_loss: 0.1682 - val_pred2_loss: 0.0799 - val_pred1_accuracy: 0.1344 - val_pred1_mean_pred: 1.6806 - val_pred2_accuracy: 0.4860 - val_pred2_mean_pred: 0.8020

Epoch 00002: val_loss improved from 0.37427 to 0.24807, saving model to ../model_weigh

137/137 [==============================] - 23s 165ms/step - loss: 0.0491 - pred1_loss: 0.0339 - pred2_loss: 0.0152 - pred1_accuracy: 0.3858 - pred1_mean_pred: 0.3392 - pred2_accuracy: 0.5635 - pred2_mean_pred: 0.1518 - val_loss: 0.0977 - val_pred1_loss: 0.0598 - val_pred2_loss: 0.0379 - val_pred1_accuracy: 0.3220 - val_pred1_mean_pred: 0.5966 - val_pred2_accuracy: 0.4644 - val_pred2_mean_pred: 0.3778

Epoch 00017: val_loss did not improve from 0.08761
Epoch 18/100
137/137 [==============================] - 23s 165ms/step - loss: 0.0446 - pred1_loss: 0.0319 - pred2_loss: 0.0127 - pred1_accuracy: 0.3869 - pred1_mean_pred: 0.3188 - pred2_accuracy: 0.5459 - pred2_mean_pred: 0.1269 - val_loss: 0.0928 - val_pred1_loss: 0.0548 - val_pred2_loss: 0.0380 - val_pred1_accuracy: 0.3444 - val_pred1_mean_pred: 0.5474 - val_pred2_accuracy: 0.4872 - val_pred2_mean_pred: 0.3824

Epoch 00018: val_loss did not improve from 0.08761
Fold 2
Build model...
Model: "model_12"
___________________________________

...

137/137 [==============================] - 23s 170ms/step - loss: 0.0426 - pred1_loss: 0.0311 - pred2_loss: 0.0115 - pred1_accuracy: 0.3699 - pred1_mean_pred: 0.3114 - pred2_accuracy: 0.5361 - pred2_mean_pred: 0.1148 - val_loss: 0.1088 - val_pred1_loss: 0.0647 - val_pred2_loss: 0.0441 - val_pred1_accuracy: 0.3728 - val_pred1_mean_pred: 0.6546 - val_pred2_accuracy: 0.4744 - val_pred2_mean_pred: 0.4496

Epoch 00017: val_loss did not improve from 0.09160
Epoch 18/100
137/137 [==============================] - 23s 171ms/step - loss: 0.0425 - pred1_loss: 0.0300 - pred2_loss: 0.0126 - pred1_accuracy: 0.3769 - pred1_mean_pred: 0.2997 - pred2_accuracy: 0.5278 - pred2_mean_pred: 0.1257 - val_loss: 0.0921 - val_pred1_loss: 0.0548 - val_pred2_loss: 0.0373 - val_pred1_accuracy: 0.3380 - val_pred1_mean_pred: 0.5556 - val_pred2_accuracy: 0.6448 - val_pred2_mean_pred: 0.3835

Epoch 00018: val_loss did not improve from 0.09160
Fold 4
Build model...
Model: "model_14"
___________________________________

137/137 [==============================] - 30s 164ms/step - loss: 0.4514 - pred1_loss: 0.2562 - pred2_loss: 0.1952 - pred1_accuracy: 0.1613 - pred1_mean_pred: 2.5621 - pred2_accuracy: 0.3502 - pred2_mean_pred: 1.9510 - val_loss: 0.3844 - val_pred1_loss: 0.2444 - val_pred2_loss: 0.1399 - val_pred1_accuracy: 0.2392 - val_pred1_mean_pred: 2.4496 - val_pred2_accuracy: 0.4952 - val_pred2_mean_pred: 1.4006

Epoch 00001: val_loss improved from inf to 0.38437, saving model to ../model_weight/V2_4.h5
Epoch 2/100
137/137 [==============================] - 23s 167ms/step - loss: 0.2294 - pred1_loss: 0.1455 - pred2_loss: 0.0839 - pred1_accuracy: 0.2090 - pred1_mean_pred: 1.4551 - pred2_accuracy: 0.4664 - pred2_mean_pred: 0.8388 - val_loss: 0.2731 - val_pred1_loss: 0.1549 - val_pred2_loss: 0.1182 - val_pred1_accuracy: 0.1800 - val_pred1_mean_pred: 1.5536 - val_pred2_accuracy: 0.3616 - val_pred2_mean_pred: 1.1809

Epoch 00002: val_loss improved from 0.38437 to 0.27309, saving model to ../model_weigh

137/137 [==============================] - 30s 162ms/step - loss: 0.4901 - pred1_loss: 0.2919 - pred2_loss: 0.1982 - pred1_accuracy: 0.1320 - pred1_mean_pred: 2.9185 - pred2_accuracy: 0.3556 - pred2_mean_pred: 1.9820 - val_loss: 0.3388 - val_pred1_loss: 0.2039 - val_pred2_loss: 0.1349 - val_pred1_accuracy: 0.0968 - val_pred1_mean_pred: 2.0296 - val_pred2_accuracy: 0.4376 - val_pred2_mean_pred: 1.3452

Epoch 00001: val_loss improved from inf to 0.33878, saving model to ../model_weight/V2_5.h5
Epoch 2/100
137/137 [==============================] - 23s 165ms/step - loss: 0.2358 - pred1_loss: 0.1495 - pred2_loss: 0.0864 - pred1_accuracy: 0.1948 - pred1_mean_pred: 1.4945 - pred2_accuracy: 0.4750 - pred2_mean_pred: 0.8635 - val_loss: 0.2553 - val_pred1_loss: 0.1582 - val_pred2_loss: 0.0972 - val_pred1_accuracy: 0.1572 - val_pred1_mean_pred: 1.5725 - val_pred2_accuracy: 0.4784 - val_pred2_mean_pred: 0.9680

Epoch 00002: val_loss improved from 0.33878 to 0.25533, saving model to ../model_weigh

137/137 [==============================] - 35s 197ms/step - loss: 0.4633 - pred1_loss: 0.2679 - pred2_loss: 0.1954 - pred1_accuracy: 0.1718 - pred1_mean_pred: 2.6760 - pred2_accuracy: 0.3593 - pred2_mean_pred: 1.9524 - val_loss: 0.4349 - val_pred1_loss: 0.2814 - val_pred2_loss: 0.1535 - val_pred1_accuracy: 0.5632 - val_pred1_mean_pred: 2.8203 - val_pred2_accuracy: 0.3692 - val_pred2_mean_pred: 1.5354

Epoch 00001: val_loss improved from inf to 0.43491, saving model to ../model_weight/V2_6.h5
Epoch 2/100
137/137 [==============================] - 23s 169ms/step - loss: 0.2521 - pred1_loss: 0.1566 - pred2_loss: 0.0955 - pred1_accuracy: 0.1882 - pred1_mean_pred: 1.5657 - pred2_accuracy: 0.4572 - pred2_mean_pred: 0.9553 - val_loss: 0.2668 - val_pred1_loss: 0.1608 - val_pred2_loss: 0.1061 - val_pred1_accuracy: 0.2148 - val_pred1_mean_pred: 1.6092 - val_pred2_accuracy: 0.4436 - val_pred2_mean_pred: 1.0587

Epoch 00002: val_loss improved from 0.43491 to 0.26684, saving model to ../model_weigh

137/137 [==============================] - 30s 161ms/step - loss: 0.4694 - pred1_loss: 0.2757 - pred2_loss: 0.1937 - pred1_accuracy: 0.1564 - pred1_mean_pred: 2.7572 - pred2_accuracy: 0.3450 - pred2_mean_pred: 1.9353 - val_loss: 0.4313 - val_pred1_loss: 0.2477 - val_pred2_loss: 0.1836 - val_pred1_accuracy: 0.0788 - val_pred1_mean_pred: 2.4772 - val_pred2_accuracy: 0.2792 - val_pred2_mean_pred: 1.8339

Epoch 00001: val_loss improved from inf to 0.43125, saving model to ../model_weight/V2_7.h5
Epoch 2/100
137/137 [==============================] - 24s 173ms/step - loss: 0.2275 - pred1_loss: 0.1451 - pred2_loss: 0.0824 - pred1_accuracy: 0.2137 - pred1_mean_pred: 1.4512 - pred2_accuracy: 0.4773 - pred2_mean_pred: 0.8240 - val_loss: 0.2828 - val_pred1_loss: 0.1751 - val_pred2_loss: 0.1077 - val_pred1_accuracy: 0.1428 - val_pred1_mean_pred: 1.7522 - val_pred2_accuracy: 0.4736 - val_pred2_mean_pred: 1.0781

Epoch 00002: val_loss improved from 0.43125 to 0.28284, saving model to ../model_weigh

In [81]:
sub=pd.DataFrame()
sub['report_ID']=test[0]
sub['Prediction']=[ '|'+' '.join(['%.12f'%j for j in i]) for i in test_oof ]
sub.to_csv('../result.csv',index=False,header=False)